In [ ]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True

In [ ]:
absoluteSize = False # True means absolute, False means relative

In [ ]:
def OptiKeyTypingTime(userKeys):
    
    timeTyping = dict()
    
    time1, t1, t2 = userKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = userKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    timeTyping['startTime'] = startTime
    timeTyping['endTime'] = endTime
    
    return timeTyping

In [ ]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [ ]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [ ]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [ ]:
def computeAggregateAverage(GazeLog, timeTyping, pupilData, subjName):
    
    typingMechanism = subjName[-2:]
    pupilData['RLCorrelation'] = list()
    
    # create a list of time of gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # Create list of pupil sizes from gazelog
    pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in GazeLog]
    pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in GazeLog]
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(timeGazeLog, timeTyping['startTime'])
    timeEnd, timeEndInd = nearestTimePoint(timeGazeLog, timeTyping['endTime'])
    
    typingPupilL = pupilLogL[timeStartInd:timeEndInd]
    typingPupilR = pupilLogR[timeStartInd:timeEndInd]
    
    # filter gaze data using hampel filter and compute average
    
    winSize = 25
    # Filter pupil sizes
    pupilHampelFilteredL = hampel(typingPupilL, winSize, 3)
    pupilHampelFilteredR = hampel(typingPupilR, winSize, 3)
    
    # compute pupil correlation and go on, only if correlation is greater than 0.5
    pupilCorr = pupilHampelFilteredL.corrwith(pupilHampelFilteredR, axis = 0)
    
    pupilData['RLCorrelation'].append(pupilCorr.values[0])
    
    print(pupilData['RLCorrelation'], 'from: ', timeStart, 'to: ', timeEnd)
    # 0.8 is a good value for correlation and was also the mean of mean of the correlations
    # for the trials for the users
    
    # Moving Mean of data without outliers: 
    pupilAbsoluteL = pupilHampelFilteredL.rolling(window=winSize, min_periods=1, center=True).mean()
    pupilAbsoluteR = pupilHampelFilteredR.rolling(window=winSize, min_periods=1, center=True).mean()
            
    # After the filtering is done: CHANGE THIS
    pupilSizeL = pupilHampelFilteredL.mean(numeric_only=float)[0]
    pupilSizeR = pupilHampelFilteredR.mean(numeric_only=float)[0] 
            
    pupilRelativeL = (pupilAbsoluteL - pupilAbsoluteL[0][0])/pupilAbsoluteL[0][0]
    pupilRelativeR = (pupilAbsoluteR - pupilAbsoluteR[0][0])/pupilAbsoluteR[0][0]
    
    if typingMechanism == 'DT':
        # add the pupil size
        if pupilData['DTFirst'] == 0:
            pupilData['DTFirst'] = 1
                
            if absoluteSize:
                pupilData['DTLeft'] = [pupilAbsoluteL.values[i][0] for i in range(0, len(pupilAbsoluteL.values))]
                pupilData['DTRight'] = [pupilAbsoluteR.values[i][0] for i in range(0, len(pupilAbsoluteR.values))]
                    
                pupilData['DTLeftSquared'] = [(pupilAbsoluteL.values[i][0])**2 for i in range(0, len(pupilAbsoluteL.values))]
                pupilData['DTRightSquared'] = [(pupilAbsoluteR.values[i][0])**2 for i in range(0, len(pupilAbsoluteR.values))]
                    
            else:
                pupilData['DTLeft'] = [pupilRelativeL.values[i][0] for i in range(0, len(pupilRelativeL.values))]
                pupilData['DTRight'] = [pupilRelativeR.values[i][0] for i in range(0, len(pupilRelativeR.values))]
                
                pupilData['DTLeftSquared'] = [(pupilRelativeL.values[i][0])**2 for i in range(0, len(pupilRelativeL.values))]
                pupilData['DTRightSquared'] = [(pupilRelativeR.values[i][0])**2 for i in range(0, len(pupilRelativeR.values))]
                    
                    
            pupilData['DTNumber'] = pupilData['DTNumber'] + 1
                
            pupilData['DTLeftMax'] = max(pupilData['DTLeft'])
            pupilData['DTRightMax'] = max(pupilData['DTRight'])
            pupilData['DTLeftMin'] = min(pupilData['DTLeft'])
            pupilData['DTRightMin'] = min(pupilData['DTRight'])
            #print(pupilData['DTNumber'])
                
        else:
            if absoluteSize:
                    
                pupilAddL = [pupilData['DTLeft'][i]+pupilAbsoluteL.values[i][0] for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['DTLeft'])))]
                pupilAddR = [pupilData['DTRight'][i]+pupilAbsoluteR.values[i][0] for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['DTRight'])))]
                    
                pupilAddSquareL = [pupilData['DTLeftSquared'][i]+(pupilAbsoluteL.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['DTLeftSquared'])))]
                pupilAddSquareR = [pupilData['DTRightSquared'][i]+(pupilAbsoluteR.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['DTRightSquared'])))]
                    
                pupilData['DTLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['DTLeftMax']])
                pupilData['DTRightMax'] = max([max(pupilAbsoluteR.values), pupilData['DTRightMax']])
                pupilData['DTLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['DTLeftMin']])
                pupilData['DTRightMin'] = min([min(pupilAbsoluteR.values), pupilData['DTRightMin']])
                
            else:
                pupilAddL = [pupilData['DTLeft'][i]+pupilRelativeL.values[i][0] for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['DTLeft'])))]
                pupilAddR = [pupilData['DTRight'][i]+pupilRelativeR.values[i][0] for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['DTRight'])))]
                
                pupilAddSquareL = [pupilData['DTLeftSquared'][i]+(pupilRelativeL.values[i][0])**2 for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['DTLeftSquared'])))]
                pupilAddSquareR = [pupilData['DTRightSquared'][i]+(pupilRelativeR.values[i][0])**2 for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['DTRightSquared'])))]
                    
                pupilData['DTLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['DTLeftMax']])
                pupilData['DTRightMax'] = max([max(pupilAbsoluteR.values), pupilData['DTRightMax']])
                pupilData['DTLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['DTLeftMin']])
                pupilData['DTRightMin'] = min([min(pupilAbsoluteR.values), pupilData['DTRightMin']])
                
            pupilData['DTLeft'] = pupilAddL
            pupilData['DTRight'] = pupilAddR
            pupilData['DTLeftSquared'] = pupilAddSquareL
            pupilData['DTRightSquared'] = pupilAddSquareR
            pupilData['DTNumber'] = pupilData['DTNumber'] + 1
                
                
                
    else:        
        if pupilData['MSFirst'] == 0:
            pupilData['MSFirst'] = 1
            if absoluteSize:
                    
                pupilData['MSLeft'] = [pupilAbsoluteL.values[i][0] for i in range(0, len(pupilAbsoluteL.values))]
                pupilData['MSRight'] = [pupilAbsoluteR.values[i][0] for i in range(0, len(pupilAbsoluteR.values))]
                    
                pupilData['MSLeftSquared'] = [(pupilAbsoluteL.values[i][0])**2 for i in range(0, len(pupilAbsoluteL.values))]
                pupilData['MSRightSquared'] = [(pupilAbsoluteR.values[i][0])**2 for i in range(0, len(pupilAbsoluteR.values))]
                    
            else:
                pupilData['MSLeft'] = [pupilRelativeL.values[i][0] for i in range(0, len(pupilRelativeL.values))]
                pupilData['MSRight'] = [pupilRelativeR.values[i][0] for i in range(0, len(pupilRelativeR.values))]
                    
                pupilData['MSLeftSquared'] = [(pupilRelativeL.values[i][0])**2 for i in range(0, len(pupilRelativeL.values))]
                pupilData['MSRightSquared'] = [(pupilRelativeR.values[i][0])**2 for i in range(0, len(pupilRelativeR.values))]
                    
                
            pupilData['MSNumber'] = pupilData['MSNumber'] + 1
                
            pupilData['MSLeftMax'] = max(pupilData['MSLeft'])
            pupilData['MSRightMax'] = max(pupilData['MSRight'])
            pupilData['MSLeftMin'] = min(pupilData['MSLeft'])
            pupilData['MSRightMin'] = min(pupilData['MSRight'])
                
                #print(subjName, 'MS')

        else:
            if absoluteSize:
                pupilAddL = [pupilData['MSLeft'][i]+pupilAbsoluteL.values[i][0] for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['MSLeft'])))]
                pupilAddR = [pupilData['MSRight'][i]+pupilAbsoluteR.values[i][0] for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['MSRight'])))]
                
                pupilAddSquareL = [pupilData['MSLeft'][i]+(pupilAbsoluteL.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['MSLeftSquared'])))]
                pupilAddSquareR = [pupilData['MSRight'][i]+(pupilAbsoluteR.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['MSRightSquared'])))]
                    
                pupilData['MSLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['MSLeftMax']])
                pupilData['MSRightMax'] = max([max(pupilAbsoluteR.values), pupilData['MSRightMax']])
                pupilData['MSLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['MSLeftMin']])
                pupilData['MSRightMin'] = min([min(pupilAbsoluteR.values), pupilData['MSRightMin']])
                    
            else:
                pupilAddL = [pupilData['MSLeft'][i]+pupilRelativeL.values[i][0] for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['MSLeft'])))]
                pupilAddR = [pupilData['MSRight'][i]+pupilRelativeR.values[i][0] for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['MSRight'])))]
                    
                pupilAddSquareL = [pupilData['MSLeftSquared'][i]+(pupilRelativeL.values[i][0])**2 for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['MSLeft'])))]
                pupilAddSquareR = [pupilData['MSRightSquared'][i]+(pupilRelativeR.values[i][0])**2 for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['MSRight'])))]
                    
                pupilData['MSLeftMax'] = max([max(pupilRelativeL.values), pupilData['MSLeftMax']])
                pupilData['MSRightMax'] = max([max(pupilRelativeR.values), pupilData['MSRightMax']])
                pupilData['MSLeftMin'] = min([min(pupilRelativeL.values), pupilData['MSLeftMin']])
                pupilData['MSRightMin'] = min([min(pupilRelativeR.values), pupilData['MSRightMin']])                    
                    
            pupilData['MSLeft'] = pupilAddL
            pupilData['MSRight'] = pupilAddR
            pupilData['MSLeftSquared'] = pupilAddSquareL
            pupilData['MSRightSquared'] = pupilAddSquareR
            pupilData['MSNumber'] = pupilData['MSNumber'] + 1
                

                
    return pupilData



In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['DTFirst'] = 0
pupilData['MSFirst'] = 0
pupilData['DTNumber'] = 0
pupilData['MSNumber'] = 0

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root or 'lone' in root or 'sc' in root or 'sh' in root or 'ae' in root or 'ma' in root or 'eo' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:
                
                # call function to check when first key is looked at, and when quit key is selected
                timeTyping = OptiKeyTypingTime(userKeys)

                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                pupilData = computeAggregateAverage(gazeLog, timeTyping, pupilData, subjName)
                
                

In [ ]:
if pupilData['DTFirst'] > 0 or pupilData['MSFirst'] > 0:
    #print(pupilData)
    # if the DT and MS data are of different sizes,
    pupilSizeMin = min(len(pupilData['DTLeft']), len(pupilData['MSLeft']))
    fig = plt.figure()
    axL = fig.add_subplot(2, 1, 1)
    axR = fig.add_subplot(2, 1, 2)
    xAxis = np.arange(0, float(pupilSizeMin/90), float(1/90))
    pupilPlot = dict()
    pupilPlot['DTLeft'] = [x/pupilData['DTNumber'] for x in pupilData['DTLeft']]
    pupilPlot['DTRight'] = [x/pupilData['DTNumber'] for x in pupilData['DTRight']]
    pupilPlot['MSLeft'] = [x/pupilData['MSNumber'] for x in pupilData['MSLeft']]
    pupilPlot['MSRight'] = [x/pupilData['MSNumber'] for x in pupilData['MSRight']]
    
    # Standard deviation
#     pupilPlot['DTLeftSquareRoot'] = [(math.fabs(pupilData['DTLeftSquared'][xInd]/pupilData['DTNumber']-(pupilPlot['DTLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['DTLeftSquared']))]
#     pupilPlot['DTRightSquareRoot'] = [(math.fabs(pupilData['DTRightSquared'][xInd]/pupilData['DTNumber']-(pupilPlot['DTRight'][xInd]))**2)**0.5 for xInd in range(0, len(pupilData['DTRightSquared']))]
#     pupilPlot['MSLeftSquareRoot'] = [(math.fabs(pupilData['MSLeftSquared'][xInd]/pupilData['MSNumber']-(pupilPlot['MSLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['MSLeftSquared']))]
#     pupilPlot['MSRightSquareRoot'] = [(math.fabs(pupilData['MSRightSquared'][xInd]/pupilData['MSNumber']-(pupilPlot['MSRight'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['MSRightSquared']))]

    pupilPlot['DTLeftSquareRoot'] = [(math.fabs((pupilData['DTLeftSquared'][xInd]-(pupilData['DTLeft'][xInd])**2)/((pupilData['DTNumber'])*(pupilData['DTNumber']-1))))**0.5 for xInd in range(0, len(pupilData['DTLeftSquared']))]
    pupilPlot['DTRightSquareRoot'] = [(math.fabs((pupilData['DTRightSquared'][xInd]-(pupilData['DTRight'][xInd])**2)/((pupilData['DTNumber'])*(pupilData['DTNumber']-1))))**0.5 for xInd in range(0, len(pupilData['DTRightSquared']))]
    pupilPlot['MSLeftSquareRoot'] = [(math.fabs((pupilData['MSLeftSquared'][xInd]-(pupilData['MSLeft'][xInd])**2)/((pupilData['MSNumber'])*(pupilData['MSNumber']-1))))**0.5 for xInd in range(0, len(pupilData['MSLeftSquared']))]
    pupilPlot['MSRightSquareRoot'] = [(math.fabs((pupilData['MSRightSquared'][xInd]-(pupilData['MSRight'][xInd])**2)/((pupilData['MSNumber'])*(pupilData['MSNumber']-1))))**0.5 for xInd in range(0, len(pupilData['MSRightSquared']))]

    pupilPlot['DTLeftStdPositive'] = [pupilPlot['DTLeft'][xInd] + pupilPlot['DTLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTLeft']))]
    pupilPlot['DTLeftStdNegative'] = [pupilPlot['DTLeft'][xInd] - pupilPlot['DTLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTLeft']))]
    pupilPlot['MSLeftStdPositive'] = [pupilPlot['MSLeft'][xInd] + pupilPlot['MSLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSLeft']))]
    pupilPlot['MSLeftStdNegative'] = [pupilPlot['MSLeft'][xInd] - pupilPlot['MSLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSLeft']))]

    pupilPlot['DTRightStdPositive'] = [pupilPlot['DTRight'][xInd] + pupilPlot['DTRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTRight']))]
    pupilPlot['DTRightStdNegative'] = [pupilPlot['DTRight'][xInd] - pupilPlot['DTRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTRight']))]
    pupilPlot['MSRightStdPositive'] = [pupilPlot['MSRight'][xInd] + pupilPlot['MSRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSRight']))]
    pupilPlot['MSRightStdNegative'] = [pupilPlot['MSRight'][xInd] - pupilPlot['MSRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSRight']))]


    axL.plot(xAxis, pupilPlot['DTLeft'][0:pupilSizeMin], 'bo', label = 'DT')
    axL.plot(xAxis, pupilPlot['MSLeft'][0:pupilSizeMin], 'ro', label = 'MS')
    axL.fill_between(xAxis, pupilPlot['DTLeftStdPositive'][0:pupilSizeMin],  pupilPlot['DTLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axL.fill_between(xAxis, pupilPlot['MSLeftStdPositive'][0:pupilSizeMin], pupilPlot['MSLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')

    
    axL.set_title('Left')
    axL.set_xlabel('Time [in s]')
    if absoluteSize:
        axL.set_ylabel('Absolute pupil size [in mm]')
    else:
        axL.set_ylabel('Relative pupil size [in %]')
    axL.legend()

    axR.plot(xAxis, pupilPlot['DTRight'][0:pupilSizeMin], 'bo', label = 'DT')
    axR.plot(xAxis, pupilPlot['MSRight'][0:pupilSizeMin], 'ro', label = 'MS')
    axR.fill_between(xAxis, pupilPlot['DTRightStdPositive'][0:pupilSizeMin], pupilPlot['DTRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axR.fill_between(xAxis, pupilPlot['MSRightStdPositive'][0:pupilSizeMin], pupilPlot['MSRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')
    
    axR.legend()
    axR.set_xlabel('Time [in s]')
    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
    else:
        axR.set_ylabel('Relative pupil size [in %]')
